「PyTorch入門  4. モデル構築」
===============================================================
【原題】BUILD THE NEURAL NETWORK

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/) 


【元URL】https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月18日

【チュトーリアル概要】

本チュートリアルでは、PyTorchでニューラルネットワークのモデルを構築する方法について解説します

---


ニューラルネットワークモデルの作り方
===================

ニューラルネットワークは、レイヤー（もしくはモジュール）と呼ばれるデータ操作の固まりで構成されています。

[`torch.nn`](https://pytorch.org/docs/stable/nn.html)で用意されているクラス、関数は、独自のニューラルネットワークを構築するために必要な要素を網羅しています。



PyTorchの全てのモジュールは、[`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承しています。


そしてニューラルネットワークは、モジュール自体が他のモジュール（レイヤー）から構成されています。

<br>

この入れ子構造により、複雑なアーキテクチャを容易に構築・管理することができます。

以下で、FashionMNIST datasetの画像データをクラス分類するネットワークモデルを構築します。




In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

---

訓練に使用するデバイス
-----------------------
可能であれば高速に訓練を行うためにも、GPU等を利用したいところです。

GPUが使用可能かどうかは、[`torch.cuda`](https://pytorch.org/docs/stable/notes/cuda.html)の結果から判断できます。

GPUが使用できない場合は、CPUで訓練を行います。



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


---



クラスの定義
-------------------------
``nn.Module``を継承し、独自のネットワークモデルを定義し、その後ネットワークのレイヤーを ``__init__``で初期化します。

``nn.Module`` を継承した全モジュールは、入力データの順伝搬関数である``forward``関数を持ちます。





In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

``NeuralNetwork``クラスのインスタンスを作成し、変数``device``上に移動させます。

以下でネットワークの構造を出力し確認します。



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを利用するには、入力データを与える必要があります。

入力データをモデルに投入すると、``forward``関数で処理されるとともに、いくつかの[`background operations`](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)が実行されます。

そのため、 ``model.forward()`` と記載して入力データを処理しないように注意してください。



今回のモデルに入力を与えると、各クラスの生の予測値を含む10次元のテンソルが返されます。

``nn.Softmax``モジュールにこの出力結果を与えることで、入力データが各クラスに属する確率の予測値を求めることができます。





In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


--------------




モデルレイヤー
-------------------------

FashionMNISTモデルを各レイヤーレベルで確認しましょう。

サイズ28x28の3枚の画像からなるミニバッチのサンプルを用意し、このミニバッチをネットワークに入力し、各処理による変化を確認していきます。



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**

[`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)レイヤーで、2次元（28x28）の画像を、1次元の784ピクセルの値へと変換します。

ミニバッチの0次元目は、サンプル番号を示す次元で、この次元は`nn.Flatten`を通しても変化しません（1次元目以降がFlattenされます）。



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear** 


[`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は、線形変換を施します。

`linear layer`は重みとバイアスのパラメータを保持しています。





In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**

非線形な活性化関数は、ニューラルネットワークの入力と出力の間にある、複雑な関係性を表現するために重要な要素です。

これらの活性化関数は線形変換のあとに、非線形性を加え、ニューラルネットワークの表現力を向上させる役割をします。

<br>

今回のモデルでは[`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)をlinear layerの間に用意しました。

なお、非線形活性化関数には他にも様々な種類があります。



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.9896, -0.1381,  0.2232,  0.6191,  0.3097, -0.2556,  0.0788, -0.6158,
          0.1954, -0.7808,  0.2459, -0.4483, -0.0637, -0.1744,  0.1014, -0.0241,
         -0.4860, -0.2461,  0.0282, -0.4358],
        [-0.7512, -0.2589,  0.2225,  0.3366,  0.1429, -0.3729,  0.0505, -0.5680,
         -0.0080, -0.3194,  0.2355, -0.5324,  0.0557, -0.3361, -0.1024, -0.2199,
         -0.2814, -0.0766,  0.0505,  0.1801],
        [-0.7155, -0.1176,  0.2091,  0.6238, -0.1321, -0.4913,  0.1382, -0.6200,
          0.1602, -0.3442,  0.4472, -0.3938, -0.1473, -0.0356, -0.3656, -0.2982,
         -0.2501, -0.0122,  0.2599, -0.0657]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2232, 0.6191, 0.3097, 0.0000, 0.0788, 0.0000, 0.1954,
         0.0000, 0.2459, 0.0000, 0.0000, 0.0000, 0.1014, 0.0000, 0.0000, 0.0000,
         0.0282, 0.0000],
        [0.0000, 0.0000, 0.2225, 0.3366, 0.1429, 0.0000, 0.0505, 0.0000, 0.0000,
         0.0000, 0.2355, 0.0000, 0.0557, 0.0000, 0.00

**nn.Sequential**

[``nn.Sequential``](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、モジュールを順番に格納する箱のような要素です。

入力データは``nn.Sequential``に定義された順番に各モジュールを伝搬します。

<br>

例えば以下の実装例のように、``seq_modules``と名付けた、複数のモジュールを束ねたモジュールを簡単に構築できます。




In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

**nn.Softmax**

ニューラルネットワークの最後のlinear layerは`logits` [- ∞, ∞] を出力します。


この`logits`は[`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)モジュールへと渡されます。


その結果、採取的な値は[0, 1]の範囲となり、これは各クラスである確率を示します。


``dim``パラメータは次元を示しており、`dim=1`の次元で和を求めると確率の総和なので1になります。




In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0956, 0.1048, 0.0904, 0.1095, 0.0941, 0.1176, 0.0985, 0.0865, 0.0976,
         0.1054],
        [0.0872, 0.0932, 0.0917, 0.1026, 0.1010, 0.1339, 0.0890, 0.0895, 0.0934,
         0.1185],
        [0.0861, 0.1006, 0.0806, 0.1065, 0.0989, 0.1385, 0.0902, 0.0855, 0.0907,
         0.1224]], grad_fn=<SoftmaxBackward0>)


---




モデルパラメータ
-------------------------

ニューラルネットワークを構成する多くのモジュールは、おのおのパラメータを保持しています。

例えば、重みやバイアスです。これらの値が訓練時に最適化されます。




``nn.Module`` を継承することで、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡でき、``parameters()`` や ``named_parameters()`` メソッドを使って、モデルの各レイヤーのすべてのパラメータにアクセスできるようになります。

以下にfor文を用いて、各パラメータを処理し、そのサイズと値を表示します。



In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0205,  0.0148, -0.0334,  ..., -0.0282,  0.0118, -0.0254],
        [-0.0199,  0.0106,  0.0250,  ..., -0.0010,  0.0083, -0.0098]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0175, 0.0335], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0173, -0.0347, -0.0182,  ...,  0.0418,  0.0027, -0.0389],
        [-0.0233, -0.0267,  0.0192,  ...,  0.0243, -0.0223,  0.0415]],
       device='cuda

--------------




さらなる詳細
--------------
以下のページも参考ください。

- [`torch.nn API`](https://pytorch.org/docs/stable/nn.html)



以上。